In [1]:
## from sympy import *
#from sympy.plotting import plot
import networkx as nx
import numpy as np
import drawSvg as draw
import time
import pprint
from itertools import count
from shapely.geometry import Polygon, Point, MultiPoint, LineString, MultiLineString
from shapely.geometry.polygon import orient
from shapely.ops import split, linemerge, nearest_points, polygonize
from shapely.affinity import rotate, translate, scale
from shapely import wkt
circle_color = "#DEB887"
river_color = "blue"
crease_color = "red"
guide_color = "lightgrey"
polygon_color = "black"
scale_factor = 100
line_thickness = 20
viewport = 700
step = 1
epsilon = 3
rounding_precision = 6
DEBUG = True

def max_val (points,x=0, y=0):
    for point in points: 
        if point.x > x: x = point.x
        if point.y > y: y = point.y
    return float(x),float(y)

def draw_poly(p,d, col):
    path = draw.Path(stroke=col, stroke_width=line_thickness/viewport*scale_factor, fill='none')
    path.M(p[0][0], p[0][1]) #point is a tuple (x,y)
    [path.L(v[0],v[1]) for v in p]
    d.append(path)
    
def render(points,circles,creases,guides,polys,rivers):
    w,h = max_val(points)
    d = draw.Drawing (w,h)
    [draw_poly(c.exterior.coords,d,circle_color) for c in circles]
    [draw_poly(p.exterior.coords,d,polygon_color) for p in polys]
    [draw_poly(c.coords,d,crease_color) for c in creases]
    [draw_poly(g.coords,d,guide_color) for g in guides]
    [draw_poly(r.coords,d, river_color) for r in rivers if type(r) == LineString]
    d.setRenderSize(viewport,viewport*h/w)
    d.saveSvg("export.svg")
    return d

def nodes_with_attribute(tree,att): return [n for n,d in tree.nodes(data=True) if d==att]
def tree_distance_from_points(d,point1,point2,ids): return d[ids[point1]][ids[point2]]
def tree_distance_from_line(d,line,ids): return tree_distance_from_points(d,line.coords[0],line.coords[-1],ids)
def get_edge_weight_from_leaf (T,node): return[(d["weight"]) for (u, v, d) in T.edges(node,data=True)][0]

def are_colinear(p1,p2,p3): return p2.intersects(line_from_points(p1,p3)) and not (p1.equals(p2) or p1.equals(p3) or p2.equals(p3))
def point_in_origin(ids,original_ids,point): return Point(point_from_id(ids[point.coords[0]],original_ids))
def point_from_id(index, ids): return Point(next((point for point, i in ids.items() if i == index), None))
def near_point_on_line(line,p): return Point(line.intersection(p.buffer(0.1)).coords[0]) 
def point_distance (ids,id1,id2): return Point(point_from_id(id1,ids)).distance(Point(point_from_id(id2,ids)))

def index_from_vertex(polygon,vertex): return polygon.exterior.coords[:].index(vertex.coords[0])
def distance_to_origin(ids,original_ids,point): return Point(point).distance(point_in_origin(ids,original_ids,point))
def buffer_line_symmetric(line_segments,offset): return [line_segments.parallel_offset(offset,'left', join_style=2),line_segments.parallel_offset(offset,'right', join_style=2)]
def azimuth(point1, point2): return np.degrees(np.arctan2(point2.x - point1.x, point2.y - point1.y))


def approximate_line(line): return wkt.loads(wkt.dumps(line,rounding_precision=rounding_precision))
def line_move_to (line,point): return translate(line, point.x- start_point(line).x,point.y - start_point(line).y)
def line_to_ids(ids,line): return ids[start_point(line).coords[0]],ids[end_point(line).coords[0]]
def line_from_ids(id1,id2,ids): return line_from_points(point_from_id(id1,ids),point_from_id(id2,ids))
def line_from_points(p1,p2): return LineString([p1,p2])
def line_stretch_to_size (line,size): return scale(line,xfact=size/line.length,yfact=size/line.length,origin=start_point(line))
def line_in_origin(ids,original_ids,line): return line_from_points(point_in_origin(ids,original_ids,start_point(line)),point_in_origin(ids,original_ids,end_point(line)))
def start_point(line): return Point(line.coords[0])
def end_point(line): return Point(line.coords[-1])

def polygon_edge(polygon,i1,i2): return LineString([polygon.exterior.coords[i1],polygon.exterior.coords[i2]])
def polygon_from_vertex(polygons,vertex): return [p for p in polygons if vertex.coords[0] in p.exterior.coords]

def tuple_in_list (a,b,l): return (a,b) in l or (b,a) in l

def project_on_origin(ids,original_ids,p1,polygon,h):
    edge = [polygon_edge(polygon,i,i+1) for i,p in enumerate(polygon.exterior.coords[:-1]) if p1.almost_equals(Point(p),rounding_precision)]
    d = line_in_origin(ids,original_ids,*edge).project(p1)
    return d if d>0 else h/2

def is_split_event(c,poly,ids,ids_origin,d,h): 
    return calculate_distances(start_point(c),end_point(c),ids,ids_origin,poly,h)-tree_distance_from_line(d,c,ids)< epsilon if not poly.touches(c) else False

def calculate_distances(p1,p2,ids,origin,poly,h): 
    
    #print(project_on_origin(ids,origin,p1,poly,h) , Point(p1).distance(Point(p2)) , project_on_origin(ids,origin,p2,poly,h), ids[p1.coords[0]],ids[p2.coords[0]])
        
    return project_on_origin(ids,origin,p1,poly,h) + Point(p1).distance(Point(p2)) + project_on_origin(ids,origin,p2,poly,h) 

def connect_points(points,lines): [extend_lines(lines,points_to_line(points.centroid,p)) for p in points.exterior.coords]

def connect_endpoints(polygon):
    shrunk_poly = polygon.buffer(-1) 
    lines = [line_stretch_to_size(line_from_points(p,shrunk_poly.exterior.coords[i+1]),3000) for i,p in enumerate(polygon.exterior.coords[1:])]
    intersection = lines[0].intersection(lines[1])
    return [line_from_points(p,intersection) for p in polygon.exterior.coords[1:]]

    
def longest_edge_length (polygon): 
    ref = 0
    for i in range(len(polygon.exterior.coords)-1):
        l = polygon_edge(polygon,i,i+1).length
        ref = l if l > ref else ref
    return ref

def path_to_leaf(node,tree,path = []):
    for n in tree.neighbors(node):
        if tree.degree(n) == 1: #found a leaf!
            return path+[n]
    path_to_leaf(n,tree,path+[n])
    
def find_colinear_points(points): return [points[i-1] for i in range(len(points)) if are_colinear(points[i-2],points[i-1],points[i])]

def adjacent_edge(polygon,vertex):
    index = index_from_vertex(polygon,vertex)
    return [polygon_edge(polygon,index,index+1), polygon_edge(polygon,index-1,index)]

def shrink_polygon(polygon,sweep_length,ids,d,active,ids_origin,h1=0,creases=[],guides=[],h=0):    
    #print ("====================STEP====================", h)
    # shrink the polygon
    h+=sweep_length
    shrunk_poly = polygon.buffer(-sweep_length) 
    
    # look for contraction events
    if shrunk_poly.is_empty: return connect_points(polygon,creases), guides 
    
    # look for split events
    if len(polygon.exterior.coords) > 4:          # ignore triangles as no split event can happen here
        # find the active paths
        a = query_matrix(active,1) 
        for point_ids in a:
            cut_path = line_from_ids(point_ids[0],point_ids[1],ids)
            if is_split_event(cut_path,polygon,ids,ids_origin,d,h1): 
                if DEBUG: print("split event", cut_path, "p1:",point_ids[0], "p2:",point_ids[1], "distance", cut_path.length)
                guides.append(cut_path)
                h1 = h
                [shrink_polygon(p,sweep_length,ids.copy(),d,get_active_paths(p,ids, d),ids.copy(),h1,creases,guides,h) for p in split(polygon, cut_path)]
                return creases,guides
    else:
        lines = connect_endpoints(polygon)
        if lines != False:            
            [creases.append(line) for line in lines]
            return creases,guides
        
    colinear = find_colinear_points(MultiPoint(polygon.exterior.coords[:-1]))    
    for point in colinear:
        points = nearest_points(point,shrunk_poly)
        proj = points[1] if points[0] == point else points[0]
        coords = shrunk_poly.exterior.coords[:]
        coords.insert(index_from_vertex(polygon,point),proj)
        if (coords[0] == proj): del coords[-1]
        shrunk_poly = Polygon(coords)
            
    # generate creases        
    for i,p in enumerate(polygon.exterior.coords):
        extend_lines(creases,LineString([p,shrunk_poly.exterior.coords[i]]))
        if i>0: ids[shrunk_poly.exterior.coords[i]] = ids.pop(p)

    # recursively call the shrinking with the new polygon
    return shrink_polygon(shrunk_poly,sweep_length,ids,d,active,ids_origin,h1,creases,guides,h)

def convex_polygon_from_points(points):
    poly = Polygon(points).convex_hull
    colinear = find_colinear_points(points)
    
def extend_lines(lines,extension):
    for i,line in enumerate(lines):
        if approximate_line(extension).touches(approximate_line(line)):
            merge = linemerge(MultiLineString([extension,line]))
            if type(merge) == LineString: lines[i] = merge
            return
    lines.append(extension)

def get_active_paths(polygon,ids,d):
    paths = np.zeros((len(ids),len(ids)))
    for p1 in polygon.exterior.coords:
        for p2 in polygon.exterior.coords:
            if ids[p1] > ids[p2]:
                paths[ids[p1]][ids[p2]] = 1 if abs(Point(p1).distance(Point(p2))-tree_distance_from_points(d,p1,p2,ids)) > epsilon else -1
    return paths

def first_intersection (line,other_lines):
    ref = float('inf')
    intersection_point = intersecting_line = None
    for other_line in other_lines:
        intersection = line.intersection(other_line)
        if not intersection.is_empty:
            if type(intersection) == Point: intersection = [intersection]
            for point in intersection:
                candidate = line_from_points(start_point(line),point)
                if epsilon < candidate.length < ref: 
                    ref = candidate.length
                    intersection_point,intersecting_line = point,other_line
    return intersection_point, intersecting_line


def propagate_line(line,creases,segments):
    intersection_point, intersecting_crease = first_intersection(line,creases)
    if intersection_point == None:      
        extend_lines(segments,line)
        return segments[-1]
    extend_lines(segments,line_from_points(start_point(line),intersection_point))
    line_slope = azimuth(start_point(line),intersection_point)
    crease_slope = azimuth(near_point_on_line(intersecting_crease,intersection_point),intersection_point)
    newline = line_move_to(rotate(line, 180+2*(line_slope - crease_slope), origin=intersection_point),intersection_point)
    return propagate_line(newline,creases, segments)
def query_matrix(m,val):
    res = np.where(m==val)
    return list(zip(res[0],res[1]))

def line_on_polygon(line,polygon): return line.relate(polygon) == 'F1FF0F212'
def line_not_in_multiline(line,lines): return line.relate(lines) == 'FF100F102'

def get_lang_polys(points,ids,d,polys=[]):
    active = get_active_paths(Polygon(points),ids,d)
    _a = query_matrix(active,-1)
    x_max,y_max = max_val(points)
    polys += [line_from_ids(point_ids[0],point_ids[1],ids) for point_ids in _a] # add all active edges to the polys
    a = query_matrix(active,1)
    inactive = [line_from_ids(point_ids[0],point_ids[1],ids) for point_ids in a] # add all active edges to the polys
    outline_edges = Polygon([point for point in points if point.x in (0,x_max) or point.y in (0,y_max)]).convex_hull
    polys += [line for line in inactive if line_on_polygon(line,outline_edges) and line_not_in_multiline(line,MultiLineString(polys))]
    return list(polygonize(polys)) if len(list(polygonize(polys))) >0 else [MultiPoint(points).convex_hull]

def generate_rivers(polygons,tree,ids,d,cr,rivers=[],visited=[]):
    innodes= nodes_with_attribute(tree,{})
    for node in innodes:        #better would be to iterate over edges I guess...
        path = path_to_leaf(node,tree)
        leaf = Point(point_from_id(path[-1],ids))
        polygon = polygon_from_vertex(polygons,leaf)[0]
        edges = adjacent_edge(polygon,Point(point_from_id(path[-1],ids)))
        for edge in edges:
            edge_path = nx.shortest_path(tree,*line_to_ids(ids,edge))
            if node in edge_path and len(edge_path)>3: 
                    e = line_from_points(leaf,end_point(edge)) if start_point(edge) == leaf else line_from_points(leaf,start_point(edge))
                    other_node = edge_path[edge_path.index(node)-1]
                    other_node = other_node if other_node in innodes else edge_path[edge_path.index(node)+1]
                    if not ((node,other_node) in visited):
                        width = d[other_node][node]/2
                        start = e.interpolate(get_edge_weight_from_leaf(tree,path[-1]) + width)
                        line = line_stretch_to_size(e,longest_edge_length(polygon))
                        line = line_move_to(rotate(line,90,origin=start),start)
                        rivers += buffer_line_symmetric(propagate_line(line,cr,[]),width)
                        rivers += buffer_line_symmetric(propagate_line(rotate(line,180,origin=start),cr,[]),width)
                        visited.extend([(other_node,node),(node,other_node)])
    return rivers


def lizardTree(T=nx.Graph(),ids={}, set_id=count()):
    #TODO: do the actual NLopt for circle packing
    raw_points = Point(0,0.6), Point(0,3.6), Point(1.908,0),Point(1.908,3), Point(3.816,0.6), Point(3.816,3.600)
    pts = [Point(p.x*scale_factor,p.y*scale_factor) for p in raw_points]
    for p in pts:
        ids[p.coords[0]] = next(set_id)
        T.add_node(ids[p.coords[0]], pos=p)
    inner_node1 = next(set_id)
    inner_node2 = next(set_id)    
    T.add_node(inner_node1)
    T.add_node(inner_node2)
    T.add_edge(0,inner_node2,weight=1*scale_factor)
    T.add_edge(1,inner_node1,weight=1*scale_factor)
    T.add_edge(2,inner_node2,weight=1*scale_factor)
    T.add_edge(3,inner_node1,weight=1*scale_factor)
    T.add_edge(4,inner_node2,weight=1*scale_factor)
    T.add_edge(5,inner_node1,weight=1*scale_factor)
    T.add_edge(inner_node1,inner_node2,weight=1*scale_factor)
    return pts,ids,T

def threeNodesTree(T=nx.Graph(),ids={}, set_id=count()):
    #TODO: do the actual NLopt for circle packing
    raw_points = Point(2,0), Point(0,0), Point(1,1.74)
    pts = [Point(p.x*scale_factor,p.y*scale_factor) for p in raw_points]
    for p in pts:
        ids[p.coords[0]] = next(set_id)
        T.add_node(ids[p.coords[0]], pos=pts)
    inner_node1 = next(set_id)
    T.add_edge(0,inner_node1,weight=1*scale_factor)
    T.add_edge(1,inner_node1,weight=1*scale_factor)
    T.add_edge(2,inner_node1,weight=1*scale_factor)
    return pts,ids,T

def antennaBeetleTree(T=nx.Graph(),ids={}, set_id=count()):
    #TODO: do the actual NLopt for circle packing
    raw_points = Point(6,2), Point(6,0), Point(4,0), Point(3,2.5), Point(6,6.12), Point(2,0), Point(0,0), Point(0,2), Point(0,6.12)
    pts = [Point(p.x*scale_factor,p.y*scale_factor) for p in raw_points]
    for p in pts:
        ids[p.coords[0]] = next(set_id)
        T.add_node(ids[p.coords[0]], pos=pts)
    inner_node1 = next(set_id)
    set_id = count()
    T.add_edge(next(set_id),inner_node1,weight=1*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=1*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=1*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=1.69*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=3*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=1*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=1*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=1*scale_factor)
    T.add_edge(next(set_id),inner_node1,weight=3*scale_factor)
    return pts,ids,T

def beetleTree(T=nx.Graph(),ids={}, set_id=count()):
    #TODO: do the actual NLopt for circle packing

    raw_points = Point(9.63,29.26), Point(0,26.56), Point(0,14.56), Point(6.63,0), Point(14.63,11.71), Point(14.63,21.94), Point(14.63,25.94), Point(19.63,29.26), Point(29.26,26.56), Point(29.26,14.56), Point(22.63,0),Point(14.63,29.26)
    pts = [Point(p.x*scale_factor,p.y*scale_factor) for p in raw_points]
    for p in pts:
        ids[p.coords[0]] = next(set_id)
        T.add_node(ids[p.coords[0]], pos=p)
    inner_node1 = next(set_id)
    inner_node2 = next(set_id)
    inner_node3 = next(set_id)
    inner_node4 = next(set_id)
    inner_node5 = next(set_id)
    inner_node6 = next(set_id) 
    
    T.add_edge(0,inner_node1,weight=4*scale_factor)
    T.add_edge(inner_node1,inner_node2,weight=1*scale_factor)
    T.add_edge(inner_node2,inner_node3,weight=1*scale_factor)
    T.add_edge(1,inner_node3,weight=4*scale_factor)
    T.add_edge(inner_node3,inner_node4,weight=1*scale_factor)
    T.add_edge(inner_node4,inner_node5,weight=1*scale_factor)
    T.add_edge(2,inner_node5,weight=6*scale_factor)
    T.add_edge(inner_node5,inner_node6,weight=2*scale_factor)   
    T.add_edge(3,inner_node6,weight=8*scale_factor)
    T.add_edge(4,inner_node6,weight=6.2*scale_factor)
    T.add_edge(5,inner_node4,weight=1*scale_factor)
    T.add_edge(6,inner_node2,weight=1*scale_factor)
    T.add_edge(7,inner_node1,weight=4*scale_factor)
    T.add_edge(8,inner_node3,weight=4*scale_factor)
    T.add_edge(9,inner_node5,weight=6*scale_factor)
    T.add_edge(10,inner_node6,weight=8*scale_factor)
    T.add_edge(11,inner_node1,weight=1*scale_factor)
    return pts,ids,T


points,point_ids,T = beetleTree()#beetleTree()#threeNodesTree()#lizardTree()#antennaBeetleTree()
tree_distances = {x[0]:x[1] for x in nx.all_pairs_dijkstra_path_length(T)}  

crease_lines=rivers=guide_lines=polys=[]
circles = [point.buffer(get_edge_weight_from_leaf(T,point_ids[point.coords[0]])) for point in points]
polys = get_lang_polys(points,point_ids,tree_distances)
for polygon in polys:
    crease_lines,guide_lines = shrink_polygon(polygon,step,point_ids.copy(),tree_distances,get_active_paths(polygon,point_ids,tree_distances),point_ids.copy())
#rivers = []   
rivers = generate_rivers(polys,T,point_ids,tree_distances,crease_lines)
render(points,circles,crease_lines,guide_lines,polys,rivers)




KeyError: (1463.194143954772, 2593.0190269500004)

600.1866376386598
600.1866376386598


In [109]:
P = Polygon([[0, 0], [1, 0], [1, 1], [0, 0]])

print(P.centroid)

POINT (0.6666666666666666 0.3333333333333333)
